In [ ]:
import pandas as pd
import numpy as np
import difflib as dl
import multiprocessing
import nltk
# nltk.download('stopwords')
from os import cpu_count
from unidecode import unidecode
from nltk.corpus import stopwords
from nltk.tokenize import wordpunct_tokenize
from nltk.stem.lancaster import LancasterStemmer

In [ ]:
year = 2013
df = pd.read_csv(f'./Raw/{year}.csv', sep=';', encoding ='latin')
df = df[['DS_RESUMO', 'DS_PALAVRA_CHAVE', 'NM_AREA_AVALIACAO']]

In [ ]:
df.columns = df.columns.str.slice(3).str.lower()
df.rename(columns={
    'base': 'ano'
}, inplace=True)

for item in list(df.iloc[:, 0:].columns):
    df[f'{item}'] = df[f'{item}'].str.lower()

In [ ]:
def rename_areas(text: str) -> str:
    if text in ['administração pública e de empresas, ciências contábeis e turismo', 'administração, ciências contábeis e turismo']:
        return 'administração pública e de empresas, ciências contábeis e turismo'
    elif text in ['arquitetura e urbanismo', 'arquitetura, urbanismo e design']:
        return 'arquitetura, urbanismo e design'
    elif text in 'filosofia/teologia:subcomissão filosofia':
        return 'filosofia'
    elif text in ['filosofia/teologia:subcomissão teologia', 'teologia']:
        return 'ciências da religião e teologia'
    elif text in 'letras / linguística':
        return 'linguística e literatura'
    elif text in 'artes / música':
        return 'artes'
    elif text in 'ciências sociais aplicadas i':
        return 'serviço social'
    else:
        return text

In [ ]:
df.area_avaliacao = df.area_avaliacao.apply(rename_areas)
del rename_areas

In [ ]:
colegios_dict = {
    'ciências da vida': {
        'ciências agrárias': ['ciência de alimentos', 'ciências agrárias i', 'medicina veterinária', 'zootecnia / recursos pesqueiros'],
        'ciências biológicas': ['biodiversidade', 'ciências biológicas i', 'ciências biológicas ii', 'ciências biológicas iii'],
        'ciências da saúde': ['educação física', 'enfermagem', 'farmácia', 'medicina i', 'medicina ii', 'medicina iii', 'nutrição', 'odontologia', 'saúde coletiva']
    },
    'humanidades': {
        'ciências humanas': ['antropologia / arqueologia', 'ciência política e relações internacionais', 'ciências da religião e teologia', 'educação', 'filosofia', 'geografia', 'história', 'psicologia', 'sociologia'],
        'ciências sociais aplicadas': ['administração pública e de empresas, ciências contábeis e turismo', 'arquitetura, urbanismo e design', 'comunicação e informação', 'direito', 'economia', 'planejamento urbano e regional / demografia', 'serviço social'],
        'linguística, letras e artes': ['artes', 'linguística e literatura']
    },
    'ciências exatas, tecnológicas e multidisciplinar': {
        'ciências exatas e da terra': ['astronomia / física', 'ciência da computação', 'geociências', 'matemática / probabilidade e estatística', 'química'],
        'engenharias': ['engenharias i', 'engenharias ii', 'engenharias iii', 'engenharias iv'],
        'multidisciplinar': ['biotecnologia', 'ciências ambientais', 'ensino', 'interdisciplinar', 'materiais']
    }
}

In [ ]:
df['colegios'] = ''
df['areas'] = ''
df.rename(columns={'area_avaliacao': 'subareas'}, inplace=True)

for colegios, areas in colegios_dict.items():
    for areas, subareas in areas.items():
        df.loc[df.subareas.isin(subareas), 'colegios'] = colegios
        df.loc[df.subareas.isin(subareas), 'areas'] = areas
        
df = df[['resumo', 'palavra_chave', 'colegios', 'areas', 'subareas']]
del colegios_dict

In [ ]:
df.palavra_chave.replace(to_replace = r'1[.]', value = ';', regex = True, inplace=True)
df.palavra_chave.replace(to_replace = r' \d[.]', value = ';', regex = True, inplace=True)
df.palavra_chave.replace(to_replace = r'(\S{2,})(?:\.)', value = '\\1;', regex=True, inplace=True)
df.palavra_chave.replace(to_replace = r'\b(pt|en)\b', value = ';', regex = True, inplace=True)
df.palavra_chave.replace(to_replace = r' ,|, | [,] | /|/ | [/] ', value = ';', regex = True, inplace=True)
df.palavra_chave.replace(to_replace = r' [.] |\(\d\)', value = ';', regex = True, inplace=True)
df.palavra_chave.replace(to_replace = r' \x96 |\x93', value = ';', regex = True, inplace=True)
df.palavra_chave.replace(to_replace = r'[^a-z0-9\s\x80-\xff,.:;?!-]', value = '', regex = True, inplace=True)
df.palavra_chave.replace(to_replace = r' - |; | ;|; ;', value = ';', regex = True, inplace=True)
df.palavra_chave.replace(to_replace = r';{2,}', value = ';', regex = True, inplace=True)
df.palavra_chave.replace(to_replace= r'[\W_]+$|^[;]+', value='', regex=True, inplace=True)
df.palavra_chave.replace(to_replace = r'palavras-chave\S', value = '', regex = True, inplace=True)
df.palavra_chave.replace(to_replace = r'^\s+|\s+$|\x80', value = '', regex = True, inplace=True)
df.palavra_chave.replace(to_replace = r'[^\w\s;]', value = '', regex = True, inplace=True)

In [ ]:
df.resumo.replace(to_replace= r'[^a-zA-Z0-9\s]+', value='', regex=True, inplace=True)

In [ ]:
def remove_stopwords(text: str) -> str:
    stop_words = set(stopwords.words('portuguese'))
    new_text = [item for item in wordpunct_tokenize(text) if item not in stop_words]
    return " ".join(new_text)

In [ ]:
def lc_stem(text: str) -> str:
    stemmer = LancasterStemmer()
    words = [stemmer.stem(word) for word in wordpunct_tokenize(text)]
    return " ".join(words)

In [ ]:
df.resumo = df.resumo.apply(remove_stopwords).apply(lc_stem).apply(lambda x: unidecode(x))
df.palavra_chave = df.palavra_chave.apply(remove_stopwords).apply(lc_stem).apply(lambda x: unidecode(x))
df.palavra_chave.replace(to_replace = r' ; ', value = ';', regex = True, inplace=True)
del remove_stopwords, lc_stem

In [ ]:
key_words = df.palavra_chave.copy()
key_words = list(set(';'.join(key_words).split(';')))
key_words = [item for item in key_words if 2 < len(item) < 80]

for i, _ in enumerate(key_words):
    key_words[i] = key_words[i].strip()

key_words = list(set(key_words))

In [ ]:
def calculate_similarity_or_pertency(sentence: str, text: str) -> str:
    if sentence in text or text in sentence:
        return True
    similaridade = dl.SequenceMatcher(None, sentence, text).ratio()
    porcentagem = int(similaridade * 100)
    if porcentagem > 75:
        return True
    
    return False

- 50 palavras_chave = 2m 42.3s
- 500 palavras_chave = 12m 24.6s
- 5000 palavras_chave = 126m 2.3s
- 109350 palavras_chave = ?

In [ ]:
def process_character(character):
    corte = df[
        df.palavra_chave.apply(
            lambda x: calculate_similarity_or_pertency(character, x)
        ) 
        |
        df.resumo.apply(
            lambda x: calculate_similarity_or_pertency(character, x)
        )
    ].iloc[:, -3:]

    results = []
    for subarea in sorted(corte.subareas.unique()):
        colegio, area = (
            corte[corte.subareas == subarea].colegios.unique()[0],
            corte[corte.subareas == subarea].areas.unique()[0],
        )
        frequencia = corte.subareas.value_counts()[subarea]
        results.append([colegio, area, subarea, character, frequencia])

    return results

max_threads = cpu_count()
data = []
with multiprocessing.Pool(processes=max_threads) as pool:
    for result in pool.imap_unordered(process_character, key_words[:500]):
        data.extend(result)

In [ ]:
columns_dtype = {
    'colegio': 'category',
    'area': 'category',
    'subarea': 'category',
    'palavra_chave': 'category',
    'frequencia': 'uint64'
}

freq = pd.DataFrame(data, columns=columns_dtype.keys()).astype(columns_dtype)

In [ ]:
freq.set_index('palavra_chave').sort_index().sort_values('frequencia', ascending=False)